In [3]:
import pickle
from pathlib import Path
import os

In [4]:
project_root = Path(os.getcwd()).parent
results_path = project_root / "results"

In [5]:
exp_name = "my_experiment"
pkl_path = results_path / exp_name / "simulation_recording.pkl"

In [6]:
with open(pkl_path, "rb") as f:
    rec = pickle.load(f)

In [7]:
print(dir(rec))

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_frame', 'frames', 'load', 'meta', 'save', 'times']


In [8]:
rec.frames

[FrameSnapshot(t=0.0, bodies={0: BodySnapshot(id=0, collider=ColliderSnapshot(kind='CircleCollider', attrs={'radius': 1.0}), pos=array([76.86020028, 44.02576317]), vel=array([7.50451196, 9.40564716]), mass=1.0, color=(0.0, 0.0, 1.0)), 1: BodySnapshot(id=1, collider=ColliderSnapshot(kind='CircleCollider', attrs={'radius': 1.0}), pos=array([10.23151077, 96.60571975]), vel=array([ 1.27840403, -3.16242592]), mass=1.0, color=(0.0, 0.0, 1.0))}, events=[]),
 FrameSnapshot(t=0.0, bodies={0: BodySnapshot(id=0, collider=ColliderSnapshot(kind='CircleCollider', attrs={'radius': 1.0}), pos=array([76.8727078 , 44.04143925]), vel=array([7.50451196, 9.40564716]), mass=1.0, color=(0.0, 0.0, 1.0)), 1: BodySnapshot(id=1, collider=ColliderSnapshot(kind='CircleCollider', attrs={'radius': 1.0}), pos=array([10.23364144, 96.60044904]), vel=array([ 1.27840403, -3.16242592]), mass=1.0, color=(0.0, 0.0, 1.0))}, events=[]),
 FrameSnapshot(t=0.0, bodies={0: BodySnapshot(id=0, collider=ColliderSnapshot(kind='Circle

In [9]:
print(rec.frames[-1])

FrameSnapshot(t=0.0, bodies={0: BodySnapshot(id=0, collider=ColliderSnapshot(kind='CircleCollider', attrs={'radius': 1.0}), pos=array([98.64978944, 72.22702859]), vel=array([-7.50451196,  9.40564716]), mass=1.0, color=(0.0, 0.0, 1.0)), 1: BodySnapshot(id=1, collider=ColliderSnapshot(kind='CircleCollider', attrs={'radius': 1.0}), pos=array([14.06459219, 87.12371269]), vel=array([ 1.27840403, -3.16242592]), mass=1.0, color=(0.0, 0.0, 1.0))}, events=[])
